# <b>Kumho Compiler</b>

### The purpose of this compiler is to:
1. Convert the DAT files into a CSV file
2. Combine multiple CSV files into one dataframe
3. Data visualization for analysis

 <br>

### 1. Converting .DAT files into .CSV file (dattocsv.py)

#### The conversion program written in python executes the following:
        -Converts the DAT files into a CSV file.
        -Saves the CSV file in a CSV folder in the parent directory

#### Requirement:
        Run the following program in a windows machine as it runs subprocess via command line switch.
https://github.com/alpacapetter/kumho-compiler

 <br>

### 2.  Multiple CSV files into one dataframe

In [1]:
import pandas as pd
import os

In [2]:
# df = pd.read_csv("/mnt/c/Users/Robert Choi/Desktop/Kumho ENG/Data/4-7-2018 DATA LOG/BM1 DROP DOOR PT-101 DATA CSV/2018 03 29 0014 (Float).csv")
files = [file for file in os.listdir("/mnt/c/Users/Rob/Documents/Datalog 2018-9-10/BM1(Q MIXER)/PT/BM1 SCRUBBER SC-101-PT-101 DATA_csv/")]

In [3]:
all_data = pd.DataFrame()

for file in files:
    df = pd.read_csv("/mnt/c/Users/Rob/Documents/Datalog 2018-9-10/BM1(Q MIXER)/PT/BM1 SCRUBBER SC-101-PT-101 DATA_csv/"+file)
    all_data = pd.concat([all_data, df])

In [ ]:
# rename columns

all_data.columns = ['Date', 'Time', 'Del1', 'value', 'Del2', 'Del3', 'Del4', 'Del5']

In [8]:
all_data = pd.DataFrame({'col':['']}).append(df, ignore_index=True)
all_data.head(1000)

,col,;Date,Time,Millitm,Tagname,Value,Status,Marker,Internal
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,14:00:02,68.0,[HKT]BM1_HMI_AI16 ...,-56.899994,,B,-1.0,NaN
2,NaN,14:00:10,373.0,[HKT]BM1_HMI_AI16 ...,-57.099991,,,0.0,NaN
3,NaN,14:00:11,724.0,[HKT]BM1_HMI_AI16 ...,-57.099991,,,1.0,NaN
4,NaN,14:00:21,680.0,[HKT]BM1_HMI_AI16 ...,-57.000000,,,2.0,NaN
...,...,...,...,...,...,...,...,...,...
358,NaN,14:59:21,718.0,[HKT]BM1_HMI_AI16 ...,-57.099991,,,356.0,NaN
359,NaN,14:59:31,675.0,[HKT]BM1_HMI_AI16 ...,-56.800003,,,357.0,NaN
360,NaN,14:59:41,685.0,[HKT]BM1_HMI_AI16 ...,-57.099991,,,358.0,NaN
361,NaN,14:59:51,696.0,[HKT]BM1_HMI_AI16 ...,-56.899994,,,359.0,NaN


Average the "Value" Parameter per date

In [ ]:
def AverageColumn (c):
    f=open(csv,"r")
    average=0
    Sum=0
    column=len(f)
    
    for i in range(0,column):
        for n in i.split(','):
            n=float(n)
            Sum += n
        average = Sum / len(column)
    return 'The average is:', average

    f.close()


csv="MDT25.csv"
print AverageColumn(csv)

 <br>

### 3.  Data visualization for analysis